In [ ]:
# default_exp covariables

# covariables

> Construcción de covariables para analizar delitos

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import requests

In [ ]:
# export
DATA_PATH = "datos/"
DOWNLOADS_PATH = "datos/descargas/"

## descarga_datos_covariables

In [ ]:
# export
def descarga_datos_covariables():
    """Descarga los archivos necesarios qe son demasiado grandes para el repositorio.
    
        - geometrías de manzanas
    """
    covariables_url = "https://www.dropbox.com/s/s49lb476wpwu2p1/covariables.gpkg?dl=1"
    r = requests.get(covariables_url, allow_redirects=True)
    open(DOWNLOADS_PATH + 'covariables.gpkg', 'wb').write(r.content)

In [ ]:
descarga_datos_covariables()

## get_diccionario_censo

In [ ]:
# export
def get_diccionario_censo():
    """Regresa un DataFrame con el diccionario de variables del censo."""
    dicionario = pd.read_csv("datos/diccionario_datos_ageb_urbana_09_cpv2020.csv", skiprows=3)
    diccionario = (dicionario
                   .drop(range(0,8))
                   .drop(columns='Núm.')
                   .reset_index(drop=True)
                   .rename({'Mnemónico':'Nombre del Campo'}, axis=1))
    return diccionario

In [ ]:
diccionario = get_diccionario_censo()
diccionario.head()

,Indicador,Descripción,Nombre del Campo,Rangos,Longitud
0,Población total,Total de personas que residen habitualmente en...,POBTOT,0...999999999,9
1,Población femenina,Total de mujeres que residen habitualmente en ...,POBFEM,0...999999999,9
2,Población masculina,Total de hombres que residen habitualmente en ...,POBMAS,0...999999999,9
3,Población de 0 a 2 años,Personas de 0 a 2 años de edad.,P_0A2,0…999999999,9
4,Población femenina de 0 a 2 años,Mujeres de 0 a 2 años de edad.,P_0A2_F,"0.,.999999999",9


## get_variables_censo

In [ ]:
# export
def get_variables_censo():
    """Regresa un DataFrame con las variables del censo a nivel manzana."""
    df = pd.read_csv("datos/censo_manzanas.zip", dtype={'CVEGEO':str})
    return df

In [ ]:
censo = get_variables_censo()
censo.head()

,CVEGEO,AMBITO,TIPOMZA,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_3YMAS,...,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,colonia_cve,cuadrante_id
0,0901000010898031,Urbana,Típica,93.0,56.0,37.0,4.0,NaN,3.0,89.0,...,15.0,16.0,6.0,3.0,0.0,0.0,7.0,0.0,1119.0,17.0
1,0901000012269024,Urbana,Típica,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082.0,14.0
2,0901000011472068,Urbana,Típica,124.0,66.0,58.0,3.0,3.0,0.0,121.0,...,25.0,22.0,9.0,8.0,0.0,NaN,7.0,0.0,1030.0,11.0
3,0901000011824024,Urbana,Típica,340.0,177.0,163.0,12.0,8.0,4.0,327.0,...,69.0,56.0,29.0,14.0,NaN,NaN,25.0,NaN,1135.0,110.0
4,0901000012377004,Urbana,Típica,82.0,41.0,41.0,NaN,0.0,NaN,80.0,...,13.0,13.0,6.0,3.0,0.0,0.0,9.0,0.0,1081.0,18.0


## censo_a_tasas

In [ ]:
def censo_a_tasas(censo, diccionario, agregacion='colonias'):
    """Convierte las variables del censo a tasas en la agregación seleccionada.

    Para las variables de población divide por población total.
    Para Hogares divide por TOTHOG.
    Para viviendas divide por el total de viviendas particulares 
     habitadas (VIVPAR_HAB)
    
    """
    pob_col = 'POBTOT'
    hog_col = 'TOTHOG'
    viv_col = 'VIVPAR_HAB'
    vars_pob = [v for v in diccionario['Nombre del Campo'].unique() if v.startswith('P')]
    eliminar = ['POBTOT', 'PROM_HNV']
    vars_pob = [v for v in vars_pob if (v not in eliminar)]
    if agregacion == None:
        censo[vars_pob] = censo[vars_pob].div(censo[pob_col], axis=0)
    return censo

In [ ]:
tasas = censo_a_tasas(censo, diccionario, None)
tasas

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [ ]:
censo.dtypes

CVEGEO            object
AMBITO            object
TIPOMZA           object
ENTIDAD           object
NOM_ENT           object
                  ...   
VPH_SINCINT       object
VPH_SINTIC        object
colonia_cve      float64
cuadrante_id      object
geometry        geometry
Length: 236, dtype: object